In [ ]:
import showdown
import agent
import asyncio
import torch
import json

In [ ]:
async def training_loop(agent, showdown, numBattles=1000):
    for battle in range(numBattles):
        
        
        await showdown.restart()
        battleDone = False
        totalReward = 0
        
        while not battleDone:
            state = showdown.getState()
            
            validActions = showdown.getValidActions()
            action = agent.act(state, validActions)
            nextState, reward, battleDone = await showdown.executeAction(action)
            print(f"Action: {action}, Reward: {reward}")
            
            agent.remember(state, action, reward, nextState, battleDone)
            agent.replay()
            totalReward += reward
            
            if battleDone:
                print(f"Battle {battle} done. Total reward: {totalReward}")
                
        
    if battle % 100 == 0:
        agent.saveModel(f"model_{battle}.pt")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
stateSize = 647

possibleActions = json.load(open("data/possible_actions.json", "r"))
actionSize = len(possibleActions)

newAgent = agent.Agent(stateSize, actionSize, device, possibleActions)

showdown = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-0", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")

await showdown.connectNoSecurity()
await training_loop(newAgent, showdown, 1000)

In [ ]:
# sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "thisisatest12345", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
# tasks = set()
# async with asyncio.TaskGroup() as tg:
#     for i in range(1):
#         sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-"+str(i), "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
#         task = tg.create_task(sd.run())
#         tasks.add(task)

# await asyncio.gather(*tg.tasks)

# await sd.run()
